<a href="https://colab.research.google.com/github/nourrosama/NTI-Tasks/blob/main/Nour_Osama_Day8_Task_(test_data)_RNN_NTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
test_data=pd.read_csv('test_data.csv')
test_data.head()

,Unnamed: 0,review_description,rating
0,22802,الثلاثاء القادم مرحبا بك يا,1
1,6087,اعجبني بصراحة وما احس الاسعار فية غالية والتوص...,1
2,26771,مافي ماكدونالدز,-1
3,15557,الطلب وصلني ناقص مع علم المطعم مع ذلك تم أخذ م...,-1
4,23820,انا عايز اشتري ابلكيشن طالبات,1


In [ ]:
#Import the necessary modules:
import pandas as pd
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet') # Download wordnet resource

stop_words_en = set(stopwords.words('english'))
stop_words_ar = set(stopwords.words("arabic"))
stop_words = stop_words_en.union(stop_words_ar)  # combine both

import emoji

def preprocess(text):
    text = str(text).lower()                        # lowercase
    text = re.sub(r'http\S+|www\S+', '', text)      # remove urls
    text = re.sub(r'@\w+', '', text)                # remove mentions
    text = re.sub(r'#\w+', '', text)                # remove hashtags

    # normalize arabic
    text = re.sub("[إأآa]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("[ًٌٍَُِّْ]", "", text)           # remove diacritics

    # keep Arabic + English letters only
    text = re.sub(r'[^a-z\u0600-\u06FF\s]', ' ', text)

    # remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    # tokenization
    words = text.split()

    # remove both arabic + english stopwords
    words = [w for w in words if w not in stop_words]

    # remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    # handle emojis
    text = emoji.demojize(text, language="en")

    # Tokenization
    tokens = word_tokenize(text)
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Stemming
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]


    # remove both arabic + english stopwords
    words = [w for w in tokens if w not in stop_words]

    return" ".join(words)


test_data['cleaned_test_data'] = test_data['review_description'].apply(preprocess)
test_data[['review_description','cleaned_test_data']].head(100)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,review_description,cleaned_test_data
0,الثلاثاء القادم مرحبا بك يا,الثلاثاء القادم مرحبا
1,اعجبني بصراحة وما احس الاسعار فية غالية والتوص...,اعجبني بصراحه احس الاسعار غاليه والتوصيل جدا رايع
2,مافي ماكدونالدز,مافي ماكدونالدز
3,الطلب وصلني ناقص مع علم المطعم مع ذلك تم أخذ م...,الطلب وصلني ناقص المطعم تم اخذ مبلغ كامل والي ...
4,انا عايز اشتري ابلكيشن طالبات,انا عايز اشتري ابلكيشن طالبات
...,...,...
95,خوه طريقه الدفع ليه اتغيرت,خوه طريقه الدفع ليه اتغيرت
96,برنامج فاشل ومعاملة مطاعم فاشله جدا بينهم وبين...,برنامج فاشل ومعامله مطاعم فاشله جدا بينهم وبين...
97,بعض الموظفين كذابين والخدمة تعبانه,الموظفين كذابين والخدمه تعبانه
98,يعطيكم الف عافيه 😍😍😍😍,يعطيكم الف عافيه


In [ ]:
X = test_data['cleaned_test_data']
y = test_data['rating']

In [ ]:
from tensorflow.keras.models import load_model
import pickle

# Load model
model = load_model("sentiment_rnn.h5")

# Load tokenizer
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)


In [ ]:
max_words = 10000   # vocabulary size
max_len = 100

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# convert labels -1, 1 → 0, 1
import numpy as np
y = np.array(y)
y = (y == 1).astype(int)

# tokenize + pad
X_test_seq = tokenizer.texts_to_sequences(X)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [ ]:
def predict(x,model):
    pred=model.predict(x)
    return pred

In [ ]:
# Evaluate
loss, acc = model.evaluate(X_test_pad, y)
print(f"Test Accuracy: {acc:.2f}")

# Or predict
pred_probs = model.predict(X_test_pad)
pred_labels = (pred_probs > 0.5).astype(int)  # threshold at 0.5


331/331 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.5969 - loss: 0.6765
Test Accuracy: 0.60
331/331 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step
